In [ ]:
import os
import re
import json
import chardet
import scr.public_function as pf

# NOTE 中间的所有文件都是UTF8
encoding = 'utf16'
output_encoding = 'utf16'

pf._init_()
name = ''

In [ ]:
# 从游戏原文本文件提取日文

def get_scenario_from_origin(data: list) -> list:
    text_all = []
    cnt = 0
    # START FIXME
    for line in data:
        if pf.has_jp(line) and line[0] not in ';[/*\t# ':
            text_all.append(line)
        cnt+=1
    # END
    return text_all

pf.extract_jp(get_scenario_from_origin, encoding)


In [5]:
# 创建翻译字典
pf.create_dict(lambda x:x[:-1])

总共 40085 条
添加关键字： 65 条


In [ ]:
# 检查字典key数量是否正确
pf.check_dict()

In [6]:
# 把字典翻译

# print(func2('「・・・ああ、うん、それで・・・スペシャル餃子、一つ・・・」'))
pf.translate(delete_func=None)
# print(pf._translate('ここは私立|最明[さいめい]学園。^n孤島に建てられ、世間と切り離された、^n全寮制のモデル学園だ。', delete_func=pf.XFL.format_xfl))

30 / 23387
60 / 23387
90 / 23387
120 / 23387
150 / 23387
180 / 23387
210 / 23387
240 / 23387
270 / 23387
300 / 23387
330 / 23387
360 / 23387
390 / 23387
420 / 23387
450 / 23387
480 / 23387
510 / 23387
540 / 23387
570 / 23387
600 / 23387
630 / 23387
660 / 23387
690 / 23387
720 / 23387
750 / 23387
780 / 23387
810 / 23387
840 / 23387
870 / 23387
900 / 23387
930 / 23387
960 / 23387
990 / 23387
1020 / 23387
1050 / 23387
1080 / 23387
1110 / 23387
1140 / 23387
1170 / 23387
1200 / 23387
1230 / 23387
1260 / 23387
1290 / 23387
1320 / 23387
1350 / 23387
1380 / 23387
1410 / 23387
1440 / 23387
1470 / 23387
1500 / 23387
1530 / 23387
1560 / 23387
1590 / 23387
1620 / 23387
1650 / 23387
1680 / 23387
1710 / 23387
1740 / 23387
1770 / 23387
1800 / 23387
1830 / 23387
1860 / 23387
1890 / 23387
1920 / 23387
1950 / 23387
1980 / 23387
2010 / 23387
2040 / 23387
2070 / 23387
2100 / 23387
2130 / 23387
2160 / 23387
2190 / 23387
2220 / 23387
2250 / 23387
2280 / 23387
2310 / 23387
2340 / 23387
2370 / 23387
2400 / 23

In [ ]:
# 检查是否还有没翻译的语句
pf.check_dict_untranslated()

In [ ]:
# 构建对比
pf.create_contrast()

In [ ]:
# 替换文本
def find(data:list, failed_text:list, jp_chs:dict):
    record = 0
    cnt = 0                         # 当前行
    start = False
    for line in data:
        #START FIXME 替换目标文本
        if pf.has_jp(line) and line[0] not in ';[/*\t# ':
            key = line[:-1]
            if key in jp_chs and jp_chs[key]:
                record += 1
                data[cnt] = data[cnt].replace(key, jp_chs[key])
            else:
                failed_text.append(key+'\n')
        cnt += 1
        # END 
    return record

pf.replace_all(find, encoding, output_encoding)

In [ ]:
name = 'scene_m0719b.spt'
data = pf.open_file_b(name)
data = bytearray(data)
i = 0
while i < len(data):
    data[i] ^= 0xFF
    i += 1
pf.save_file_b('new_'+name, data)

In [ ]:
bin(0xff)